<a href="https://colab.research.google.com/github/shiwangupadhyay/5_min_XAUUSD_strategy/blob/main/5_min_XAUUSD_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies

In [ ]:
pip install ta backtesting==0.5.0 bokeh==2.4.3

importing dependencies

In [ ]:
import requests
import pandas as pd
import ta
from backtesting import Backtest, Strategy

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Importing OHLC data for backtesting using Polygon.io API

In [78]:
API_KEY = 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ'

def get_xauusd_data(api_key, multiplier=5, timespan="minute", from_date="2024-09-01", to_date="2024-12-31"):
    url = f"https://api.polygon.io/v2/aggs/ticker/C:XAUUSD/range/{multiplier}/{timespan}/{from_date}/{to_date}"

    params = {
        "apiKey": 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ',
        "limit": 100000
    }

    all_data = []
    next_url = url

    while next_url:
        response = requests.get(next_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            next_url = data.get("next_url")
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
        return df[['timestamp', 'o', 'h', 'l', 'c', 'v']]
    else:
        return None

xauusd_data_5min = get_xauusd_data(API_KEY)
if xauusd_data_5min is not None:
    print(xauusd_data_5min.head())

            timestamp        o        h        l        c   v
0 2024-09-01 21:00:00  2502.98  2502.98  2502.98  2502.98   1
1 2024-09-01 21:05:00  2505.05  2505.05  2505.05  2505.05   1
2 2024-09-01 22:00:00  2501.57  2502.06  2500.96  2501.99   9
3 2024-09-01 22:05:00  2502.37  2503.62  2501.30  2503.62  12
4 2024-09-01 22:10:00  2504.23  2504.48  2503.20  2503.98  11


Renaming columns

In [79]:

df = xauusd_data_5min.rename(columns={'timestamp': 'Time', 'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v' : 'Volume'})
df.tail(10)

,Time,Open,High,Low,Close,Volume
23207,2024-12-31 17:25:00,2626.11,2626.22,2624.60,2625.24,14
23208,2024-12-31 17:30:00,2624.94,2625.24,2623.86,2623.86,14
23209,2024-12-31 17:35:00,2624.06,2624.06,2623.26,2623.56,13
23210,2024-12-31 17:40:00,2623.96,2624.36,2623.07,2623.07,13
23211,2024-12-31 17:45:00,2623.40,2623.99,2623.07,2623.99,13
23212,2024-12-31 17:50:00,2624.12,2624.82,2623.50,2624.82,14
23213,2024-12-31 17:55:00,2624.15,2624.98,2624.14,2624.14,13
23214,2024-12-31 18:00:00,2623.21,2624.92,2623.15,2624.92,13
23215,2024-12-31 18:05:00,2624.19,2624.19,2624.19,2624.19,6
23216,2024-12-31 22:00:00,2624.19,2624.19,2624.19,2624.19,1


adding RSI and EMA column

In [80]:
df.set_index('Time', inplace = True)


df['Rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

df['Ema9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()


df.dropna(inplace=True)

df

,Open,High,Low,Close,Volume,Rsi,Ema9
Time,,,,,,,
2024-09-01 22:55:00,2503.72,2503.72,2502.50,2503.18,12,48.402084,2503.452659
2024-09-01 23:00:00,2503.28,2503.43,2502.00,2503.10,12,47.718447,2503.382127
2024-09-01 23:05:00,2503.12,2503.12,2501.70,2502.72,11,44.503088,2503.249702
2024-09-01 23:10:00,2502.72,2502.78,2501.40,2501.98,12,38.992953,2502.995761
2024-09-01 23:15:00,2502.37,2502.45,2501.30,2502.12,11,40.494066,2502.820609
...,...,...,...,...,...,...,...
2024-12-31 17:50:00,2624.12,2624.82,2623.50,2624.82,14,55.639627,2624.527705
2024-12-31 17:55:00,2624.15,2624.98,2624.14,2624.14,13,52.566768,2624.450164
2024-12-31 18:00:00,2623.21,2624.92,2623.15,2624.92,13,55.596120,2624.544131


Backtesting strategy for given data using backtesting.py

In [81]:

class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Rsi[-1] < self.rsi_low and
            self.data.Close[-1] < self.data.Ema9[-1] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Rsi[-1] > self.rsi_high and
              self.data.Close[-1] > self.data.Ema9[-1] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [20]
rsi_low_list = [20,30]
rsi_high_list = [60,70,80]
sl_pips_list = [20]
pip_difference_list = [15]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'CAGR [%]': stats['CAGR [%]'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '30T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:136: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:153: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  trades = trades.assign(count=1).resample(freq, on='Exit

Printing parameters for best result

In [82]:
best_result

,0
target_pips,20.000000
sl_pips,20.000000
rsi_low,20.000000
rsi_high,60.000000
pip_difference,15.000000
Final Equity,32885.530000
# Trades,396.000000
CAGR [%],1094.200781
Win Rate [%],53.030303
Sharpe Ratio,0.021195


In [83]:
results_df = results_df.sort_values(by='Win Rate [%]', ascending=False)
results_df.head(50)

,target_pips,sl_pips,rsi_low,rsi_high,pip_difference,Final Equity,# Trades,CAGR [%],Win Rate [%],Sharpe Ratio,Return,Max. Drawdown [%],Avg. Drawdown [%],Worst Trade [%],Best Trade [%]
1,20,20,20,70,15,28524.53,160,787.887033,53.750000,0.130993,185.2453,-60.777939,-13.872544,-0.126345,0.138772
0,20,20,20,60,15,32885.53,396,1094.200781,53.030303,0.021195,228.8553,-78.020554,-16.310217,-0.136200,0.148106
3,20,20,30,60,15,1785.56,533,-97.238023,48.780488,-0.478355,-82.1444,-94.368475,-17.182295,-0.160916,0.148106
2,20,20,20,80,15,13306.67,33,81.331824,48.484848,0.427863,33.0667,-25.957732,-13.672385,-0.104452,0.138772
4,20,20,30,70,15,1539.66,297,-97.971569,45.791246,-5.045647,-84.6034,-95.875363,-12.542255,-0.160916,0.139910
5,20,20,30,80,15,729.62,170,-99.571958,38.823529,-123.883338,-92.7038,-97.359274,-16.063889,-0.160916,0.139910
